<a href="https://colab.research.google.com/github/ayushksingh28/torch_itt/blob/main/torch_ffnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
import torch

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [30]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x






In [31]:
#model = NN(784, 10)
#x = torch.randn(64, 784)
#print(model(x).shape)

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [33]:
input_size = 784
num_classes = 10
learning_rate = 0.001
num_epochs = 3
batch_size = 64

In [34]:
train_dataset = datasets.MNIST(root = "datasets/", train = True, transform = transforms.ToTensor(), download = True)

In [35]:
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle =True)

In [36]:
test_dataset = datasets.MNIST(root = "datasets/", train = False, transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle =True)

In [37]:
model = NN(input_size = input_size, num_classes = num_classes).to(device)

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [39]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Reshape the data tensor
        data = data.view(data.size(0), -1)

        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()


In [40]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'got (num_correct)/(num_samples) with accuracy (float(num_correct)/float(num_samples)*100:.2f)')
  model.train()
  return num_correct / num_samples

In [41]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

got (num_correct)/(num_samples) with accuracy (float(num_correct)/float(num_samples)*100:.2f)
Accuracy on training set: 96.27
got (num_correct)/(num_samples) with accuracy (float(num_correct)/float(num_samples)*100:.2f)
Accuracy on test set: 95.70
